In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame,Series

In [3]:
df = DataFrame({'key1':['a','b','c','a','c'],
               'key2':['one','two','one','two','three'],
               'data1':np.random.randn(5),
               'data2':np.random.randn(5)})
df

,data1,data2,key1,key2
0,1.023629,2.087165,a,one
1,0.789319,0.728076,b,two
2,-0.982533,-0.319293,c,one
3,0.216595,-0.473751,a,two
4,-0.127350,0.349500,c,three


按 key1 进行分组并计算 data1 的平均值，实现方式很多，这里用： 访问 data1，并根据 key1 调用 groupby。

In [4]:
grouped = df['data1'].groupby(df['key1'])
grouped

变量 grouped 是一个 Groupby 对象，它实际上没有进行任何计算，但已经包含了执行运算所需的一切。

In [5]:
#计算平均值
grouped.mean()

key1
a    0.620112
b    0.789319
c   -0.554942
Name: data1, dtype: float64

如果一次传入多个数组，会得到如下结果：

In [9]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2 
a     one      1.023629
      two      0.216595
b     two      0.789319
c     one     -0.982533
      three   -0.127350
Name: data1, dtype: float64

In [10]:
means.unstack()

key2,one,three,two
key1,,,
a,1.023629,NaN,0.216595
b,NaN,NaN,0.789319
c,-0.982533,-0.12735,NaN


- 实际上分组键可以是任意长度适合的数组，如下：

In [13]:
import numpy as np
stats = np.array(['A','B','A','A','B'])
years = np.array([2005,2006,2006,2006,2005])
df['data1'].groupby([stats,years]).mean()

A  2005    1.023629
   2006   -0.382969
B  2005   -0.127350
   2006    0.789319
Name: data1, dtype: float64

此外，你还可以把列名用作分组键：

In [14]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.620112,0.806707
b,0.789319,0.728076
c,-0.554942,0.015104


可以看到key2列没有，这是因为该列不是数值，俗称‘麻烦列’，所以从结果中排除了。

In [15]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                     
a    one    1.023629  2.087165
     two    0.216595 -0.473751
b    two    0.789319  0.728076
c    one   -0.982533 -0.319293
     three -0.127350  0.349500

- size方法，返回一个包含分组大小的Series。

In [16]:
df.groupby(['key1','key2']).size()

key1  key2 
a     one      1
      two      1
b     two      1
c     one      1
      three    1
dtype: int64

In [17]:
df.groupby('key1').size()

key1
a    2
b    1
c    2
dtype: int64

## 对分组进行迭代

In [18]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0  1.023629  2.087165    a  one
3  0.216595 -0.473751    a  two
b
      data1     data2 key1 key2
1  0.789319  0.728076    b  two
c
      data1     data2 key1   key2
2 -0.982533 -0.319293    c    one
4 -0.127350  0.349500    c  three


In [21]:
for (k1,k2),group in df.groupby(['key1','key2']):
    print(k1,k2)
    print(group)

a one
      data1     data2 key1 key2
0  1.023629  2.087165    a  one
a two
      data1     data2 key1 key2
3  0.216595 -0.473751    a  two
b two
      data1     data2 key1 key2
1  0.789319  0.728076    b  two
c one
      data1     data2 key1 key2
2 -0.982533 -0.319293    c  one
c three
     data1   data2 key1   key2
4 -0.12735  0.3495    c  three


- 有一个可能有用的运算，将数据片段做成一个字典：

In [22]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':       data1     data2 key1 key2
 0  1.023629  2.087165    a  one
 3  0.216595 -0.473751    a  two, 'b':       data1     data2 key1 key2
 1  0.789319  0.728076    b  two, 'c':       data1     data2 key1   key2
 2 -0.982533 -0.319293    c    one
 4 -0.127350  0.349500    c  three}

In [23]:
pieces['a']

,data1,data2,key1,key2
0,1.023629,2.087165,a,one
3,0.216595,-0.473751,a,two


- groupby 默认是在axis=0上进行分组，也可以在其他任何轴上分组，下面根据dtype对列分组：

In [24]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [25]:
grouped = df.groupby(df.dtypes,axis=1)

In [26]:
dict(list(grouped))

{dtype('float64'):       data1     data2
 0  1.023629  2.087165
 1  0.789319  0.728076
 2 -0.982533 -0.319293
 3  0.216595 -0.473751
 4 -0.127350  0.349500, dtype('O'):   key1   key2
 0    a    one
 1    b    two
 2    c    one
 3    a    two
 4    c  three}

- 语法糖

    选取部分列进行聚合，尤其对于大数据量，很可能只需对部分列进行聚合：

In [28]:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

是以下代码的语法糖：

In [29]:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

## 通过字典或Series进行分组

In [30]:
people = DataFrame(np.random.randn(5,5),
                  columns=['a','b','c','d','e'],
                  index=['Joe', 'Steve', 'Wes', 'Jim', 'Tom'])


In [31]:
people.ix[2:3, ['b','c']]=np.nan
people

,a,b,c,d,e
Joe,-1.265614,0.086115,1.216777,0.715948,0.742495
Steve,1.385583,0.879546,0.328880,0.349437,0.286163
Wes,0.959472,NaN,NaN,-0.363566,2.373446
Jim,0.311447,0.324140,-0.935236,0.480040,0.395466
Tom,0.576234,0.272281,-0.271944,-0.192782,-0.208079


In [32]:
mapping = {'a':'red','b':'red','c':'blue', 'd':'blue', 'e':'red', 'f':'orange'}

In [33]:
by_column = people.groupby(mapping,axis=1)

In [34]:
by_column.sum()

,blue,red
Joe,1.932725,-0.437004
Steve,0.678317,2.551292
Wes,-0.363566,3.332918
Jim,-0.455196,1.031053
Tom,-0.464726,0.640435


In [35]:
by_column.count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Tom,2,3


### 通过函数进行分组

In [41]:
people

,a,b,c,d,e
Joe,-1.265614,0.086115,1.216777,0.715948,0.742495
Steve,1.385583,0.879546,0.328880,0.349437,0.286163
Wes,0.959472,NaN,NaN,-0.363566,2.373446
Jim,0.311447,0.324140,-0.935236,0.480040,0.395466
Tom,0.576234,0.272281,-0.271944,-0.192782,-0.208079


In [42]:
#按名字长度分组
people.groupby(len).sum()

,a,b,c,d,e
3,0.581539,0.682536,0.009597,0.639640,3.303328
5,1.385583,0.879546,0.328880,0.349437,0.286163


## 数据聚合

In [43]:
df

,data1,data2,key1,key2
0,1.023629,2.087165,a,one
1,0.789319,0.728076,b,two
2,-0.982533,-0.319293,c,one
3,0.216595,-0.473751,a,two
4,-0.127350,0.349500,c,three


In [14]:
def peak_to_pead(arr):
    return arr.max() - arr.min()

In [45]:
grouped = df.groupby('key1')

In [46]:
grouped.agg(peak_to_pead)

,data1,data2
key1,,
a,0.807034,2.560915
b,0.000000,0.000000
c,0.855183,0.668793


**注意：自定义函数要比自带的慢得多**

## 面向列的多函数应用

In [2]:
import pandas as pd

In [3]:
tips = pd.read_csv('/sun/bigdata/data/tips.csv')

In [5]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [6]:
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [7]:
grouped = tips.groupby(['sex','smoker'])

In [10]:
grouped_pct = grouped['tip_pct']

In [11]:
grouped_pct.mean()

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

In [12]:
grouped_pct.agg('mean')

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

In [15]:
#多函数
grouped_pct.agg(['mean','std', peak_to_pead])

mean       std  peak_to_pead
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

In [16]:
#指定列名
grouped_pct.agg([('foo','mean'),('bar','std')])

foo       bar
sex    smoker                    
Female No      0.156921  0.036421
       Yes     0.182150  0.071595
Male   No      0.160669  0.041849
       Yes     0.152771  0.090588

In [18]:
#对指定列执行指定运算
functions = ['count', 'max', 'mean']
grouped['tip_pct','total_bill'].agg(functions)

tip_pct                     total_bill                  
                count       max      mean      count    max       mean
sex    smoker                                                         
Female No          54  0.252672  0.156921         54  35.83  18.105185
       Yes         33  0.416667  0.182150         33  44.30  17.977879
Male   No          97  0.291990  0.160669         97  48.33  19.791237
       Yes         60  0.710345  0.152771         60  50.81  22.284500

In [19]:
functions = [('foo','count'), ('bar','max')]  #这里也可以指定列名
grouped['tip_pct','total_bill'].agg(functions)

tip_pct           total_bill       
                  foo       bar        foo    bar
sex    smoker                                    
Female No          54  0.252672         54  35.83
       Yes         33  0.416667         33  44.30
Male   No          97  0.291990         97  48.33
       Yes         60  0.710345         60  50.81

- 对不同的列应用不同的函数

In [21]:
grouped.agg({'size':'max','tip':'sum'})

size     tip
sex    smoker              
Female No         6  149.77
       Yes        4   96.74
Male   No         6  302.00
       Yes        5  183.07

- 无索引

In [22]:
tips.groupby(['sex','smoker'], as_index=False).mean()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771


In [23]:
tips.groupby(['sex','smoker'], ).mean()

total_bill       tip      size   tip_pct
sex    smoker                                          
Female No       18.105185  2.773519  2.592593  0.156921
       Yes      17.977879  2.931515  2.242424  0.182150
Male   No       19.791237  3.113402  2.711340  0.160669
       Yes      22.284500  3.051167  2.500000  0.152771

### 分组级运算和转换

In [27]:
from pandas import DataFrame,Series
import numpy as np
people = DataFrame(np.random.randn(5,5),
                  columns=['a','b','c','d','e'],
                  index=['Joe', 'Steve', 'Wes', 'Jim', 'Tom'])

In [28]:
people

,a,b,c,d,e
Joe,0.976571,0.218140,0.033487,0.022447,-0.328645
Steve,-0.898253,-0.030619,-0.449794,2.422999,1.458055
Wes,-0.670133,-1.012886,0.330553,0.340931,-1.563332
Jim,0.699725,0.507666,0.092751,-0.062108,-0.267298
Tom,1.209889,-0.381051,1.899696,0.828658,-0.699097


In [29]:
key = ['one','two','one','two','one']
people.groupby(key).transform(np.mean)

,a,b,c,d,e
Joe,0.505442,-0.391932,0.754579,0.397345,-0.863692
Steve,-0.099264,0.238524,-0.178522,1.180446,0.595378
Wes,0.505442,-0.391932,0.754579,0.397345,-0.863692
Jim,-0.099264,0.238524,-0.178522,1.180446,0.595378
Tom,0.505442,-0.391932,0.754579,0.397345,-0.863692


In [30]:
people.groupby(key).mean()

,a,b,c,d,e
one,0.505442,-0.391932,0.754579,0.397345,-0.863692
two,-0.099264,0.238524,-0.178522,1.180446,0.595378


- 分组距平

In [31]:
def demean(arr):
    return arr - arr.mean()

In [32]:
demeaned = people.groupby(key).transform(demean)
demeaned

,a,b,c,d,e
Joe,0.471129,0.610072,-0.721091,-0.374898,0.535046
Steve,-0.798989,-0.269142,-0.271273,1.242554,0.862677
Wes,-1.175575,-0.620954,-0.424026,-0.056414,-0.699640
Jim,0.798989,0.269142,0.271273,-1.242554,-0.862677
Tom,0.704446,0.010882,1.145117,0.431313,0.164594


In [33]:
#检查距平之后均值是否为0
demeaned.groupby(key).mean()

,a,b,c,d,e
one,-7.401487e-17,1.850372e-17,-7.401487e-17,0.0,7.401487e-17
two,0.000000e+00,0.000000e+00,0.000000e+00,0.0,-5.551115e-17


### apply 一般性的“拆分-应用-合并”

apply会将待处理的对象拆分成多个片段，然后对各片段调用传入的函数，最后尝试将各片段合并到一起。

In [36]:
def top(df,n=5,column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [37]:
top(tips,n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [38]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [41]:
tips.groupby('smoker').apply(top,n=3,column='tip')  #这里top的参数可以写在apply里面。

total_bill    tip     sex smoker  day    time  size   tip_pct
smoker                                                                   
No     59        48.27   6.73    Male     No  Sat  Dinner     4  0.139424
       23        39.42   7.58    Male     No  Sat  Dinner     4  0.192288
       212       48.33   9.00    Male     No  Sat  Dinner     4  0.186220
Yes    214       28.17   6.50  Female    Yes  Sat  Dinner     3  0.230742
       183       23.17   6.50    Male    Yes  Sun  Dinner     4  0.280535
       170       50.81  10.00    Male    Yes  Sat  Dinner     3  0.196812

禁止分组键

In [42]:
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


## 用特定于分组的值填充缺失数据

In [43]:
s = Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.408584
2         NaN
3   -0.921256
4         NaN
5    1.147023
dtype: float64

In [45]:
#用平均值填充NaN
s.fillna(s.mean())

0   -0.060939
1   -0.408584
2   -0.060939
3   -0.921256
4   -0.060939
5    1.147023
dtype: float64

- 对不同的分组填充不同的值

In [50]:
stats = ['Ohio','NY','Vermont','Florida','Oregan','Nevada','California','Idaha']
group_key = ['EAST']*4 + ['WEST']*4
data = Series(np.random.randn(8), index=stats)
data[['Vermont','Nevada','Idaha']] = np.nan
data

Ohio         -0.555160
NY            1.051712
Vermont            NaN
Florida       0.961770
Oregan       -0.084624
Nevada             NaN
California    1.922582
Idaha              NaN
dtype: float64

In [51]:
data.groupby(group_key).mean()   #计算均值会自动过滤掉Nan

EAST    0.486107
WEST    0.918979
dtype: float64

用分组的均值去填充Nan

In [52]:
fill_mean = lambda g:g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)  #这里的分组键可以是任意长度适合的数组，apply方法会自动应用到每个value上。

Ohio         -0.555160
NY            1.051712
Vermont       0.486107
Florida       0.961770
Oregan       -0.084624
Nevada        0.918979
California    1.922582
Idaha         0.918979
dtype: float64

用预定义的值填充

In [53]:
fill_values = {'EAST':2.2,'WEST':3.3}
fill_func = lambda g:g.fillna(fill_values[g.name])  #利用了分组的name属性
data.groupby(group_key).apply(fill_func)

Ohio         -0.555160
NY            1.051712
Vermont       2.200000
Florida       0.961770
Oregan       -0.084624
Nevada        3.300000
California    1.922582
Idaha         3.300000
dtype: float64

## 实例：随机采样和排列

扑克牌

In [63]:
suits = ['H','S','C','D']
card_val = (list(range(1,11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2,11)) + ['J','K','Q']
cards = []

In [64]:
for suit in suits:
    cards.extend(str(n) + suit for n in base_names)

In [68]:
deck = Series(card_val,index=cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

现在有了一个长度52的Series，索引为牌名，值是点数。

从整副牌抽5张：

In [69]:
def draw(deck,n=5):
    return deck.take(np.random.permutation(len(deck))[:n])

In [70]:
draw(deck)

JC    10
QH    10
9S     9
8C     8
5C     5
dtype: int64

每个花色选2张：

In [72]:
get_suit = lambda card:card[-1]
deck.groupby(get_suit).apply(draw,n=2)

C  4C      4
   6C      6
D  10D    10
   QD     10
H  10H    10
   QH     10
S  8S      8
   9S      9
dtype: int64

In [73]:
deck.groupby(get_suit,group_keys=False).apply(draw,n=2)

3C      3
2C      2
6D      6
3D      3
10H    10
6H      6
2S      2
3S      3
dtype: int64

### 分组加权平均数和相关系数

含有分组键，值以及权重值：

In [75]:
df = DataFrame({'category':['a','a','a','a','b','b','b','b'],
               'data':np.random.randn(8),
               'weight':np.random.rand(8)})
df

,category,data,weight
0,a,0.157611,0.527704
1,a,0.341824,0.247001
2,a,1.231602,0.000328
3,a,-0.027608,0.652065
4,b,-1.497144,0.045503
5,b,0.225052,0.209845
6,b,1.668308,0.615897
7,b,-1.257468,0.041341


In [76]:
np.average(range(1,5))

2.5

In [78]:
np.average(range(1,5),weights=[10,1,1,1])  # 过程：1 x 10/13 + 2 x 1/13 + 3 x 1/13 + 4 x 1/13

1.4615384615384615

计算分组加权平均值：

In [80]:
grouped = df.groupby('category')
get_wavg = lambda g:np.average(g['data'], weights=g['weight'])
grouped.apply(get_wavg)

category
a    0.105112
b    1.046064
dtype: float64

## 透视表和交叉表

In [88]:
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587


In [87]:
tips.pivot_table(columns=['sex','smoker']).unstack()   #pivot_table 默认计算平均数

smoker                    No        Yes
           sex                         
total_bill Female  18.105185  17.977879
           Male    19.791237  22.284500
tip        Female   2.773519   2.931515
           Male     3.113402   3.051167
size       Female   2.592593   2.242424
           Male     2.711340   2.500000
tip_pct    Female   0.156921   0.182150
           Male     0.160669   0.152771

In [102]:
tips.pivot_table(['tip_pct','size'],columns=['sex','day',], ).unstack()

day                  Fri       Sat       Sun      Thur
        sex                                           
tip_pct Female  0.199388  0.156470  0.181569  0.157525
        Male    0.143385  0.151577  0.162344  0.165276
size    Female  2.111111  2.250000  2.944444  2.468750
        Male    2.100000  2.644068  2.810345  2.433333

In [103]:
tips.pivot_table(['tip_pct','size'],columns=['sex','day','smoker'], ).unstack()

smoker                     No       Yes
        sex    day                     
tip_pct Female Fri   0.165296  0.209129
               Sat   0.147993  0.163817
               Sun   0.165710  0.237075
               Thur  0.155971  0.163073
        Male   Fri   0.138005  0.144730
               Sat   0.162132  0.139067
               Sun   0.158291  0.173964
               Thur  0.165706  0.164417
size    Female Fri   2.500000  2.000000
               Sat   2.307692  2.200000
               Sun   3.071429  2.500000
               Thur  2.480000  2.428571
        Male   Fri   2.000000  2.125000
               Sat   2.656250  2.629630
               Sun   2.883721  2.600000
               Thur  2.500000  2.300000

### 交叉表

In [107]:
pd.crosstab([tips.time,tips.day], tips.smoker,margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244